In [1]:
# Install necessary libraries
# ! pip install gensim
# ! pip install numpy scipy
# ! pip install pyLDAvis

In [2]:
# Import necessary libraries
import os
import nltk
nltk.download('wordnet')
from nltk.corpus import stopwords
from wordcloud import WordCloud
import matplotlib.pyplot as plt

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

import gensim
from gensim import corpora, models
from gensim.utils import simple_preprocess
from gensim.models import LdaModel
from gensim.models import CoherenceModel
from gensim.models.ldamodel import LdaModel
from gensim.models.coherencemodel import CoherenceModel

import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/manishrawat/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


### 1. Preprocessing and exploratory data analysis

We will need to load the data from the provided folder into your Python environment. We can use the os library to navigate through the folder and load the text files. Once we have loaded the data, we can build a corpus by splitting the input data into individual articles and appending them to a list.

In [3]:
# Navigate to the folder containing the data
folder_path = 'Articles'

# Load the text files into a list
articles = []
for filename in os.listdir(folder_path):
    with open(os.path.join(folder_path, filename), 'r', encoding='utf-8') as f:
        text = f.read()
        articles.append(text)
        
# Create a corpus object containing all the articles
corpus = ' '.join(articles)

Now that we have a corpus, we will need to clean it up by separating meta-data from the actual articles. 

We can do this by using regular expressions to match and remove any headers or tags that appear in the text.

In [4]:
import re

# Define regular expressions to match meta-data
header_pattern = re.compile(r'(?m)^[A-Z ]+:')
tag_pattern = re.compile(r'\[[A-Z ]+\]')

# Remove meta-data from the corpus
corpus = header_pattern.sub('', corpus)
corpus = tag_pattern.sub('', corpus)

Next, we will need to extract features from the corpus. This involves tokenizing the text and removing any stop words or words that appear too frequently or infrequently. 

We can use the **nltk library** to tokenize the text and filter out stop words, and the **gensim** library to remove words that appear too frequently or infrequently.